# This notebook assumes that SQLite database has been set up locally

In [24]:
# Hack for now to import from relative path
import sys
sys.path.append("..")

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from src.utils.llm_utils import extract_xml
from prompts.prompts_v1 import sql_generation_prompt_template, sql_generation_prompt_template_basic

import boto3
from langchain.llms.bedrock import Bedrock

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Intialize LLM and LLMChain

In [18]:
# Intialize bedrock LLM
bedrock_runtime = boto3.client("bedrock-runtime")
llm = Bedrock(client=bedrock_runtime, model_id="anthropic.claude-v2")

# Create chains based on custom prompt
sql_gen_prompt_template = PromptTemplate(
    input_variables=["USER_QUESTION"],
    template=sql_generation_prompt_template_basic,
)

llm_chain_basic = LLMChain(llm=llm, prompt=sql_gen_prompt_template)

In [19]:
response = llm_chain_basic("Count the number of rows in the database")

In [25]:
generated_sql = extract_xml(["generated_sql", 'sql'], response['text'])

In [26]:
generated_sql

'SELECT COUNT(*) FROM tablename; '

## Connect to DB and execute against DB. Log the results. 

In [27]:
import sqlite3
import pandas as pd

In [32]:
DB_PATH = "../database/synpuf/synpuf.db"

#create db connection
con = sqlite3.connect(DB_PATH)

In [33]:
df = pd.read_sql('SELECT COUNT(*) FROM outpatient_claims', con)

In [34]:
json_result = df.to_json(orient='records')
dict_result = df.to_dict(orient='records')

In [37]:
json_result, dict_result

('[{"COUNT(*)":790790}]', [{'COUNT(*)': 790790}])

In [38]:
con.close()

## Save the generated sql to txt

In [52]:
def save_strings_to_file(string_list, file_name):
    """Save a list of strings to a file, each string on a new line."""
    with open(file_name, 'w') as file:
        for string in string_list:
            file.write(string + '\n')

## Create gold sql txt

In [40]:
def save_gold_strings_to_file(string_list, db_id, file_name):
    """Save a list of strings to a file, each string on a new line."""
    with open(file_name, 'w') as file:
        for string in string_list:
            file.write(string + "\t" + db_id + '\n')

In [42]:
ground_truth_df = pd.read_csv("s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined_20231114.csv")
goal_sql_list = list(ground_truth_df.query_corrected_sqlite_with_list)
save_gold_strings_to_file(goal_sql_list, "synpuf", "../outputs/synpuf_gold_sql.txt")

## Generate and save SQL itera

In [75]:
ground_truth_df = pd.read_csv("s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined_20231114.csv")

user_query_list = list(ground_truth_df.question_with_list) # Fill in additional questions user wants to ask
generated_sql_list = []
sql_exec_result_list = []

DB_PATH = "../merck-text2sql-genai-poc/database/synpuf.db"
#create db connection
con = sqlite3.connect(DB_PATH)

# Intialize bedrock LLM
bedrock_runtime = boto3.client("bedrock-runtime")
llm = Bedrock(client=bedrock_runtime, model_id="anthropic.claude-v2")

# Create chains based on custom prompt
sql_gen_prompt_template = PromptTemplate(
    input_variables=["USER_QUESTION"],
    template=sql_generation_prompt_template_basic, # Use the right prompt template
)

llm_chain = LLMChain(llm=llm, prompt=sql_gen_prompt_template)

for query in user_query_list:
    try:
        llm_response = llm_chain(query)
        generated_sql = extract_xml("generated_sql", llm_response['text'])[0][:-1]
    except:
        # Default SQL query if failed to extract SQL from LLM response
        generated_sql = "SELECT COUNT(*) FROM outpatient_claims"
        
    generated_sql_list.append(generated_sql)
    
    # sql_exec_result = pd.read_sql(generated_sql, con)
    # json_result = sql_exec_result.to_json(orient='records')
    # sql_exec_result_list.append(json_result)
    
generated_sql_output_path = "outputs/synpuf_pred_sql.txt" # Fill in path to save the generated sql
save_strings_to_file(generated_sql_list, generated_sql_output_path)

# Close DB connection
con.close()

count the number of outpatients
SELECT COUNT(*)FROM outpatient_dat
count the number of inpatients
"SELECT COUNT(*) FROM patients WHERE inpatient = 1;
select all inpatients in year 2005
SELECT * FROM inpatientsWHERE EXTRACT(YEAR FROM admission_date) = 200
select all outpatients in year 2006
SELECT * FROM outpatients WHERE year = 200
return total patients from 2015 to 2016
SELECT COUNT(*) AS total_patientsFROM patientsWHERE year BETWEEN 2015 AND 201
select the inpatients who receive hospitalization because of diabetes of code list (4580, 7866, 6186, 29590, 5849)
SELECT COUNT(*) FROM outpatient_claims
select patients who lived though 30 years
SELECT * FROM patientsWHERE age >= 3
select patients who died at most 30 years old
SELECT * FROM patientsWHERE age_at_death <= 3
select the outpatients who are diagnosed with diabetes of code list ('V5841', 'V5832', '9594', '78943', '6009')
SELECT *FROM Outpatients WHERE DiagnosisCode IN ('V5841', 'V5832', '9594', '78943', '6009'
select all male outp